**Imports**

In [67]:
import h5py
#
from sklearn.model_selection import train_test_split

#Evaluaiton du modèle
from sklearn.metrics import classification_report

import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F

#Scheduler pour le learning rate
from torch.optim.lr_scheduler import StepLR

#Entrainement du modèle
import torch.optim as optim

#Visualisation des résultats
from torch.utils.tensorboard import SummaryWriter

#Fonction de perte
from sklearn.utils.class_weight import compute_class_weight


Chargement des datasets

In [68]:
dataset_augmented_path = 'D:/ApprentissageMachineTPFinal/data/dog_dataset_aug_normal.h5'

**Classe pour charger le dataset aug**

In [69]:
# Classe personnalisée pour charger les données
class DogBreedDataset(Dataset):
    def __init__(self, dataset_augmented_path, transform=None):
        self.file_path = dataset_augmented_path
        self.transform = transform
        
        # Charger uniquement les dimensions
        with h5py.File(self.file_path, "r") as f:
            if "images" in f:
                self.data_len = f["images"].shape[0] # type: ignore
            else:
                raise KeyError("Dataset 'images' not found in the HDF5 file.")
        
    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        with h5py.File(self.file_path, "r") as f:
            image = f["images"][idx] / 255.0  # type: ignore # Normalisation
            label = f["labels"][idx] # type: ignore
        
        # Appliquer une transformation éventuelle
        if self.transform:
            image = self.transform(image)

        # Format PyTorch
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

# Initialisation des datasets
dataset = DogBreedDataset(dataset_augmented_path)

# Division en ensembles
indices = list(range(len(dataset)))
train_indices, temp_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

# Création des DataLoaders
train_loader = DataLoader(Subset(dataset, train_indices), batch_size=64, shuffle=True)
val_loader = DataLoader(Subset(dataset, val_indices), batch_size=64, shuffle=False)
test_loader = DataLoader(Subset(dataset, test_indices), batch_size=64, shuffle=False)

**Creation du modele**

In [70]:
class DogBreedCNN(nn.Module):
    def __init__(self, num_classes=120):
        super(DogBreedCNN, self).__init__()

        # Convolutional layers with BatchNorm and ReLU
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Reduce spatial dimensions by half
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(0.5)


    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)

        # Flatten the output
        x = x.view(x.size(0), -1)
        #print(f"Shape after flattening: {x.shape}")

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        # Fully connected layers
        x = self.fc2(x)

        return x

model = DogBreedCNN(num_classes=120)

Test manuel d'etape

In [71]:
#images, labels = next(iter(train_loader))
#outputs = model(images)
#print(f"Model outputs shape: {outputs.shape}")

Vérification le contenu des données dans le DataLoader

In [72]:
for images, labels in train_loader:
    print(f"Images shape: {images.shape}")
    print(f"Sample images shape: {images.shape}")
    print(f"Sample labels: {labels}")
    break

Images shape: torch.Size([64, 3, 224, 224])
Sample images shape: torch.Size([64, 3, 224, 224])
Sample labels: tensor([ 37, 102, 100,  14,  46,  63,  19,  76,  96,   6,  19,  21,  64, 109,
        114, 101,  59,  93,  87, 111,  33,   9,  14, 104,  40,   7,  52,  91,
         32,  42,  54,  20, 110,   8, 118,  26,  10,  19,  68,  72,  75,  17,
         54,  63,  39,  34,  10,  13,  33, 107,  67,  53,  75,  22,  56,  28,
         34,  59,  62,  60,   8,  17,  57, 101])


**Entrainement aug**

In [ ]:
# Configurations
learning_rate = 0.0001
num_epochs = 30

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    scheduler.step()


Epoch [1/10], Loss: 4.7961
Epoch [2/10], Loss: 4.6905
Epoch [3/10], Loss: 4.5744
Epoch [4/10], Loss: 4.4842
Epoch [5/10], Loss: 4.4223
Epoch [6/10], Loss: 4.3845
Epoch [7/10], Loss: 4.3549
Epoch [8/10], Loss: 4.3303
Epoch [9/10], Loss: 4.3015
Epoch [10/10], Loss: 4.2884


**Evaluation du modele**

In [74]:
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Rapport de classification
print(classification_report(y_true, y_pred, zero_division=1))

              precision    recall  f1-score   support

           0       0.01      0.03      0.02        62
           1       1.00      0.00      0.00        27
           2       0.07      0.06      0.06        51
           3       0.00      0.00      0.00        54
           4       1.00      0.00      0.00        56
           5       0.05      0.06      0.06        54
           6       1.00      0.00      0.00        22
           7       0.12      0.49      0.19        70
           8       0.37      0.15      0.21        47
           9       0.04      0.02      0.03        47
          10       0.01      0.02      0.01        49
          11       0.19      0.06      0.09        49
          12       1.00      0.00      0.00        20
          13       1.00      0.00      0.00        60
          14       0.00      0.00      0.00        38
          15       1.00      0.00      0.00        46
          16       0.00      0.00      0.00        51
          17       1.00    

Visualisez les Résultats

In [77]:
val_loss=0.0
writer = SummaryWriter()
for epoch in range(num_epochs):
    writer.add_scalar('Loss/train', running_loss, epoch)
    writer.add_scalar('Loss/val', val_loss, epoch)
writer.close()
